# Método de Ramificación y acotamiento 

In [6]:
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog, minimize, differential_evolution
from dsplot.tree import BinaryTree

In [68]:
def sub(number):
    subscript_map = {
        "0": "₀",
        "1": "₁",
        "2": "₂",
        "3": "₃",
        "4": "₄",
        "5": "₅",
        "6": "₆",
        "7": "₇",
        "8": "₈",
        "9": "₉",
        "i": "ᵢ",
        }

    return ''.join(subscript_map[digit] for digit in str(number))


# Clase que implementa los problemas de ramificación y acotamiento 
class Planteamiento:
    def __init__(self, funcion_objetivo: List, restricciones_desigualdad: List, restricciones_igualdad: List = [], 
                 modo = "max", tipo="entero", variables_enteras: List= [], variables_continuas: List = []):
        if tipo not in ["entero", "mixto", "binario"]:
            raise Exception(f"{modo} No es un tipo de problema valido.")
        self.modo = "min"
        self.tipo = tipo
        self.funcion_objetivo = np.array(funcion_objetivo)
        self.variables_enteras = variables_enteras
        self.variables_continuas = variables_continuas
        

        # El solver utilizado minimiza por default, si queremos maximizar hay que multiplicar la funcion objetivo por -1
        if modo == "max":
            self.modo = "max"
            self.funcion_objetivo *= -1
        
        self.restricciones_desigualdad = np.array([restriccion[:-1] for restriccion in restricciones_desigualdad])
        self.valor_restricciones_desigualdad = np.array([valor[-1] for valor in restricciones_desigualdad])
        
        if restricciones_igualdad:
            self.restricciones_igualdad = np.array([restriccion[:-1] for restriccion in restricciones_igualdad])
            self.valor_restricciones_igualdad = np.array([valor[-1] for valor in restricciones_igualdad])
        else:
            self.restricciones_igualdad = []
            self.valor_restricciones_igualdad = []
        
        self.solucion: OptimizeResult or None = None
        
        if tipo == "binario":
            numero_variables = len(self.funcion_objetivo)
            for i in range(numero_variables):
                temp = [0.0] * numero_variables
                temp[i] = 1.0
                self.restricciones_desigualdad = np.concatenate((self.restricciones_desigualdad, np.array([temp])), axis=0)
                self.valor_restricciones_desigualdad = np.concatenate((self.valor_restricciones_desigualdad, np.array([1.0])), axis=0)


    def __str__(self) -> str:
        modelo = ""
        
        # Formato a la función objetivo
        if self.modo == "min":
            modelo += "Minimizar\nz = "
            for i, x in enumerate(self.funcion_objetivo, start=1):
                modelo += f"{x}x{sub(i)}"
                try:
                    if funcion_objetivo[i] >= 0:
                        modelo += "+"
                except IndexError:
                    continue
        else:
            modelo += "Maximizar\nz = "
            for i, x in enumerate(self.funcion_objetivo, start=1):
                modelo += f"{-x}x{sub(i)}"
                try:
                    if funcion_objetivo[i] >= 0:
                        modelo += "+"
                except IndexError:
                    continue
        
        modelo += "\n\nSujeto a:\n\n"

        # Formato a las restricciones
        for j, restriccion in enumerate(self.restricciones_desigualdad):
            for i, x in enumerate(restriccion, start=1):
                modelo += f"{x}x{sub(i)}"
                try:
                    if restriccion[i] >= 0:
                        modelo += "+"
                except IndexError:
                    continue
            modelo += f" <= {self.valor_restricciones_desigualdad[j]}\n"
            
        if self.tipo == "entero":
            modelo += f"x{sub('i')} ∈ ℤ"

        elif self.tipo == "mixto":
            
            modelo += "x"
            for var in self.variables_continuas:
                modelo += f"{sub(var)},"
            modelo += "\b ∈ ℝ\n"

            modelo += "x"
            for var in self.variables_enteras:
                modelo += f"{sub(var)},"
            modelo += "\b ∈ ℤ"

        elif self.tipo == "binario":
            modelo += f"x{sub('i')} ∈ " + "{0,1}"
        return modelo
    

    def solve(self):
        if self.restricciones_igualdad:
            self.solucion = linprog(self.funcion_objetivo, A_ub=self.restricciones_desigualdad, b_ub=self.valor_restricciones_desigualdad,
                                     A_eq=self.restricciones_igualdad, b_eq=self.valor_restricciones_igualdad)
        else:
            self.solucion = linprog(self.funcion_objetivo, A_ub=self.restricciones_desigualdad, b_ub=self.valor_restricciones_desigualdad)


In [69]:
funcion_objetivo = [3.0, 1.0, 3.0]

Restricciones = [[-1.0, 2.0, 1.0, 4.0],
                 [0.0, 4.0, -3.0, 2.0],
                 [1.0, -3.0, 2.0, 3.0]]

ejemplo =  Planteamiento(funcion_objetivo=funcion_objetivo, restricciones_desigualdad=Restricciones)
ejemplo.solve()
print(ejemplo)
print(ejemplo.solucion)

Maximizar
z = 3.0x₁+1.0x₂+3.0x₃

Sujeto a:

-1.0x₁+2.0x₂+1.0x₃ <= 4.0
0.0x₁+4.0x₂-3.0x₃ <= 2.0
1.0x₁-3.0x₂+2.0x₃ <= 3.0
xᵢ ∈ ℤ
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -29.0
              x: [ 5.333e+00  3.000e+00  3.333e+00]
            nit: 3
          lower:  residual: [ 5.333e+00  3.000e+00  3.333e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-2.000e+00 -3.000e+00 -5.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
